In [16]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import os
from vnstock import company_officers, stock_historical_data

In [17]:
symbol = input("Symbol = ")

start_date = input(f"Start date = (YYYY-MM-DD)").strip()


to_date = input(f"To date = (YYYY-MM-DD)").strip()

interval = input("""interval = 
                 
                ONE_MINUTE = "1"
                FIVE_MINUTES = "5"
                FIFTEEN_MINUTES = "15"
                THIRTY_MINUTES = "30"

                ONE_HOUR = "1H"

                ONE_DAY = "1D"
                ONE_WEEK = "1W"
                ONE_MONTH = "1M"
                 """).strip()
is_get_now = input("Get now? (y/n)").strip().lower()[0] == "y"

In [18]:
df_from_vnstock = stock_historical_data(symbol=symbol, start_date=start_date, end_date=to_date, type='stock', beautify=True, decor=False, source='DNSE')

In [19]:
df_from_vnstock

,time,open,high,low,close,volume,ticker
0,2024-01-02,27150,27250,27050,27200,3300000,CTG
1,2024-01-03,27200,27600,27050,27600,3123600,CTG
2,2024-01-04,27650,29500,27550,28600,14693100,CTG
3,2024-01-05,28750,29150,28700,29000,8765200,CTG
4,2024-01-08,29500,29800,29050,29550,12113500,CTG
...,...,...,...,...,...,...,...
220,2024-11-20,33050,34000,33050,34000,6494000,CTG
221,2024-11-21,34000,35000,33600,35000,7810800,CTG
222,2024-11-22,34900,35200,34700,35000,6181600,CTG
223,2024-11-25,34950,35100,34700,35100,3824500,CTG


In [20]:
df_from_vnstock.rename(
    columns={
        "time": "Date",
        "close": "Close",
        "open": "Open",
        "high": "High",
        "low": "Low",
        "volume": "Volume"
    },
    inplace=True
)
df_from_vnstock = df_from_vnstock[[
    "Date",
    "Close",
    "Open",
    "High",
    "Low",
    "Volume",
]]
df_from_vnstock['Date'] = pd.to_datetime(df_from_vnstock['Date']).dt.strftime('%m-%d-%Y 00:00:00+00:00')
df_from_vnstock['Date'] = pd.to_datetime(df_from_vnstock['Date'])


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_35420\417069179.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_from_vnstock['Date'] = pd.to_datetime(df_from_vnstock['Date']).dt.strftime('%m-%d-%Y 00:00:00+00:00')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_35420\417069179.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_from_vnstock['Date'] = pd.to_datetime(df_from_vnstock['Date'])


In [21]:
df_from_vnstock["Open"] = df_from_vnstock["Open"] / 1000
df_from_vnstock["Close"] = df_from_vnstock["Close"] / 1000
df_from_vnstock["High"] = df_from_vnstock["High"] / 1000
df_from_vnstock["Low"] = df_from_vnstock["Low"] / 1000

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_35420\1569936571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_from_vnstock["Open"] = df_from_vnstock["Open"] / 1000
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_35420\1569936571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_from_vnstock["Close"] = df_from_vnstock["Close"] / 1000
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_35420\1569936571.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [22]:
df_from_vnstock["Date"]

0     2024-01-02 00:00:00+00:00
1     2024-01-03 00:00:00+00:00
2     2024-01-04 00:00:00+00:00
3     2024-01-05 00:00:00+00:00
4     2024-01-08 00:00:00+00:00
                 ...           
220   2024-11-20 00:00:00+00:00
221   2024-11-21 00:00:00+00:00
222   2024-11-22 00:00:00+00:00
223   2024-11-25 00:00:00+00:00
224   2024-11-26 00:00:00+00:00
Name: Date, Length: 225, dtype: datetime64[ns, UTC]

In [23]:
now = datetime.now().date()
if not is_get_now:
    df_from_vnstock = df_from_vnstock[df_from_vnstock["Date"] != str(now)]

In [24]:
symbol = symbol.replace(".", "_").lower()
symbol = symbol.lower()
interval = interval.lower()
folder_save = f"../../data/{symbol}"
os.makedirs(folder_save, exist_ok=True)
file_name = f"{folder_save}/{symbol}_to_now_{interval}.csv"


old_df = pd.read_csv(f"{folder_save}/{symbol}_to_now_{interval}.csv")
old_df["Date"] = pd.to_datetime(old_df["Date"])

In [25]:
old_df = old_df[['Open', 'High',
                'Low', 'Close',     
                'Volume', 'Date']]

In [26]:
max_date_in_old_df = str(old_df["Date"].max())

In [27]:
df_today = df_from_vnstock[df_from_vnstock["Date"] > max_date_in_old_df]

In [28]:
old_df["Date"] = old_df["Date"].dt.date
df_today["Date"] = df_today["Date"].dt.date
df = pd.concat([old_df, df_today], ignore_index=True)
df["Date"] = pd.to_datetime(df["Date"])

In [29]:
df

,Open,High,Low,Close,Volume,Date
0,10.47,11.77,10.47,10.49,2401900,2009-07-16
1,10.10,10.20,9.97,9.97,1002480,2009-07-17
2,9.47,9.47,9.47,9.47,156560,2009-07-20
3,9.00,9.29,9.00,9.13,2133390,2009-07-21
4,9.29,9.39,9.16,9.26,949090,2009-07-22
...,...,...,...,...,...,...
3826,33.05,34.00,33.05,34.00,6494000,2024-11-20
3827,34.00,35.00,33.60,35.00,7810800,2024-11-21
3828,34.90,35.20,34.70,35.00,6181600,2024-11-22
3829,34.95,35.10,34.70,35.10,3824500,2024-11-25


In [30]:
file_name = file_name.replace("-", "_")

In [31]:
file_name

'../../data/ctg/ctg_to_now_1d.csv'

In [32]:
df_from_vnstock.to_csv(file_name, index=False)